In [1]:
!pip install torch==1.8.1
!pip install torchtext==0.9.1

     |████████████████████████████████| 804.1 MB 2.1 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.1
    Uninstalling torch-1.9.1:
      Successfully uninstalled torch-1.9.1
     |████████████████████████████████| 7.1 MB 4.0 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.1
    Uninstalling torchtext-0.10.1:
      Successfully uninstalled torchtext-0.10.1


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
from numpy import array
from nltk.tree import Tree
import re

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchtext
from torchtext.legacy import data
from torchtext.legacy.data import Field
from torchtext.legacy.data import TabularDataset, BucketIterator

import random
import math
import time

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# file processing
en_file = open('../input/koenen/ko-en.en.parse.syn.txt','r', encoding='UTF8')
english = en_file.readlines() 

ko_File = open('../input/koenko/ko-en.ko.parse.txt','r', encoding='UTF8')
korean = ko_File.readlines()

In [4]:
# function to read raw text file
def read_text(filename): 
    # open the file 
    file = open('../input/koenko/ko-en.ko.parse.txt', mode='rt', encoding='utf-8') 
        
    # read all text 
    text = file.read() 
    file.close() 
    return text
    
# split text into sentences # 이 노트북에서는 형태소로 나뉜 자료로 사용했습니다.
def to_lines(text): 
    sents = text.strip().split('\n\n')  # 한 문장 내 분리된 채 사용할 때
    return sents

def to_other_lines(text): 
    sents = text.strip().split('\n\n') 
    sents = [i.split('\t') for i in sents]  #이거 입력하면 어절별로 처리 가능
    return sents

In [5]:
# 데이터프레임으로 만들기(한국어 문장)
data = read_text("ko-en.ko.parse.txt") 
ko_eng_lines = to_lines(data) 
ko_eng = pd.DataFrame(ko_eng_lines, columns = ["read_lines"])

pd.set_option('display.max_colwidth', 250)
ko_eng.head()

,read_lines
0,<id 1>\n<sent 1>\n1\t2\tNP\t777/SN\n2\t6\tNP_SBJ\t항공편/NNG|은/JX\n3\t4\tNP\t1/SN|시간/NNG\n4\t6\tNP_AJT\t동안/NNG\n5\t6\tNP_AJT\t지상/NNG|에/JKB\n6\t7\tVP\t머물/VV|게/EC\n7\t0\tVP\t되/VV|ㅂ니다/EF|./SF\n</sent>\n</id>
1,<id 2>\n<sent 1>\n1\t2\tNP\t777/SN\n2\t6\tNP_SBJ\t항공편/NNG|은/JX\n3\t4\tNP\t3/SN|시간/NNG\n4\t6\tNP_AJT\t동안/NNG\n5\t6\tNP_AJT\t지상/NNG|에/JKB\n6\t0\tVP\t있/VV|겠/EP|습니다/EF|./SF\n</sent>\n</id>
2,"<id 3>\n<sent 1>\n1\t2\tNP\t1/SN|,/SP|000/SN|달러/NNB\n2\t3\tNP_SBJ\t여행자/NNG|수표/NNG|가/JKS\n3\t0\tVP\t필요/NNG|하/XSA|ㅂ니다/EF|./SF\n</sent>\n</id>"
3,"<id 4>\n<sent 1>\n1\t3\tNP_SBJ\t1/SN|,/SP|250/SN|원/NNB|이/JKS\n2\t3\tNP\t공식/NNG\n3\t0\tVNP\t환율/NNG|이/VCP|ㅂ니다/EF|./SF\n</sent>\n</id>"
4,<id 5>\n<sent 1>\n1\t2\tNP\t100/SN|달러/NNB\n2\t3\tNP_CNJ\t3/SN|장/NNB|과/JC\n3\t5\tNP_SBJ\t나머지/NNG|는/JX\n4\t5\tNP_AJT\t20/SN|달러/NNB|권/XSN|으로/JKB\n5\t0\tVP\t주/VV|시/EP|ㅂ시오/EF|./SF\n</sent>\n</id>


In [6]:
# 데이터프레임으로 만들기(영어 문장)
sentences = []
for a in english:
    first_list = []
    seconde_list = []
    t = Tree.fromstring(a)
    t.pos()
    for i in t.pos():
        a,b = i
        a += ' '
        b += ' '
        first_list.append(a)
        seconde_list.append(b)
        row = (first_list, seconde_list)
    sentences.append((''.join(row[0]), ''.join(row[1])))
    
english_df = pd.DataFrame(sentences, columns = ["English_list", "English_POS"])
english_df.head()

,English_list,English_POS
0,Flight 007 will stay on the ground for one hour .,NNP NNP MD VB IN DT NN IN CD NN .
1,Flight 017 will stay on the ground for three hours .,NNP NNP MD VB IN DT NN IN CD NNS .
2,"I need 1,000 dollars in traveler 's checks .",PRP VBP CD NNS IN NN POS NNS .
3,"The official exchange rate is around 1,250 Won .",DT JJ NN NN VBZ IN NNP NNP .
4,Please give me three hundred dollar bills and twenty dollar bills for the rest .,UH VB PRP CD CD NN NNS CC CD NN NNS IN DT NN .


In [7]:
# 병렬 데이터를 만들기 위한 기본 전처리
def basic_cleaning_ko(text):
    text = re.sub(r'[A-Z0-9_]+\t', '', text) # 탭과 결합된 문자 제거
    text = re.sub(r'<id [0-9]+>', '', text) # 불필요한 글자 제거
    text = re.sub(r'</id>', '', text)
    text = re.sub(r'<sent [0-9]+>', '', text)
    text = re.sub(r'</sent>', '', text)
    text = re.sub(r'\n', ' ', text) # 줄바꿈 공백으로 바꿔주기
    text = re.sub(r'\|', ' ', text) # vertical var도 공백으로 바꿔주기
    text = re.findall('(.+?)/[A-Z]+\s*', text) # 품사기호 앞에 있는 단어만 추출
    return ' '.join(text)

def basic_cleaning_en(text):
    text = re.split('(\d+)', text) # 숫자와 글자 분리
    return ' '.join(text)

In [8]:
ko_eng["clean_Korean"] = ko_eng["read_lines"].apply(basic_cleaning_ko)
english_df["clean_English"] = english_df["English_list"].apply(basic_cleaning_en)

In [9]:
# 한국어-영어 병렬 데이터 만들기
ko_trans =ko_eng["clean_Korean"]
en_trans = english_df["clean_English"]
final_df = pd.concat([ko_trans, en_trans], axis=1)
final_df

,clean_Korean,clean_English
0,777 항공편 은 1 시간 동안 지상 에 머물 게 되 ㅂ니다 .,Flight 007 will stay on the ground for one hour .
1,777 항공편 은 3 시간 동안 지상 에 있 겠 습니다 .,Flight 017 will stay on the ground for three hours .
2,"1 , 000 달러 여행자 수표 가 필요 하 ㅂ니다 .","I need 1 , 000 dollars in traveler 's checks ."
3,"1 , 250 원 이 공식 환율 이 ㅂ니다 .","The official exchange rate is around 1 , 250 Won ."
4,100 달러 3 장 과 나머지 는 20 달러 권 으로 주 시 ㅂ시오 .,Please give me three hundred dollar bills and twenty dollar bills for the rest .
...,...,...
330969,"저 의 주소 가 지방 으로 되 어 있 는데 , 서울 에서 여권 을 만들 ㄹ 수 있 나요 ?",Can I file my passport application in Seoul even if I do n't live in that area ?
330970,학교 친구 한 명 이 LA 에 있 어요 .,I have a school friend in Los Angeles .
330971,이것 ㄴ 얼마 이 에요 ?,How much is this ?
330972,저것 ㄴ 얼마 이 에요 ?,How much is that ?


In [10]:
# 토큰화 
def tok_english(text): 
    text = text.lower() # 소문자 변환
    text = text.replace("n't", 'not') # 기본적인 축약형 수정
    text = text.replace("'ve", 'have')
    text = text.replace("'m", 'am')
    text = text.replace("'s", ' ')
    text = text.replace("'ll", 'will')
    text = text.replace("'d", 'would')    
    #text = re.sub(r'[^\w\s]', '', text) # 구두점 제거
    text = text.replace('  ', ' ') # 다중 공백 제거
    text = text.replace('  ', ' ') # 두번
    return ''.join(text).split() # 단어 분리

def tok_korean(text):
   # text = re.sub(r'[^\w\s]', '', text) # 구두점 제거 
    text = text.replace('  ', ' ') # 다중 공백 제거
    return ''.join(text).split() # 단어 분리

In [11]:
# devide into train and test set

SEED = 42

from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(final_df, test_size=0.2, shuffle=True, random_state=SEED)

In [12]:
print(len(train_data))
print(len(test_data))

264779
66195


In [13]:
# save to csv file
train_data.to_csv("train.csv")
test_data.to_csv("test.csv")

In [14]:
train_reading = pd.read_csv("./train.csv")
train_reading.head(1)

,Unnamed: 0,clean_Korean,clean_English
0,41455,이 전화 로 스위스 에 걸 ㄹ 수 있 습니까 ?,Can I call Switzerland with this telephone ?


In [15]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Transformer

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchtext
from torchtext.legacy import data
from torchtext.legacy.data import Field
from torchtext.legacy.data import TabularDataset, BucketIterator

In [18]:
SRC = Field(tokenize = tok_korean, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

TRG = Field(tokenize = tok_english, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True,
            batch_first = True)

Raw_datafields = [("Unnamed:0", None), ("src", SRC), ("trg", TRG)]

train_data = data.TabularDataset(
        path='./train.csv',
        format='csv',
        skip_header=True, 
        fields=Raw_datafields) 
test_data = data.TabularDataset(
        path='./test.csv',
        format='csv',
        skip_header=True,
        fields=Raw_datafields)

In [19]:
SRC.build_vocab(train_data, min_freq = 1)
TRG.build_vocab(train_data, min_freq = 1)

BATCH_SIZE = 64

train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data), sort = False, repeat = False,
    sort_within_batch =False,
     batch_size = BATCH_SIZE,
     device = device)

In [20]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()

        self.device = device        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        batch_size = src.shape[0]
        src_len = src.shape[1]        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        return src
    
class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        _src, _ = self.self_attention(src, src, src, src_mask)        
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        _src = self.positionwise_feedforward(src)
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        return src
    
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)        
        self.dropout = nn.Dropout(dropout)        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)                
        x = torch.matmul(self.dropout(attention), V)        
        x = x.permute(0, 2, 1, 3).contiguous()        
        x = x.view(batch_size, -1, self.hid_dim)        
        x = self.fc_o(x)
        
        return x, attention
    
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        x = self.dropout(torch.relu(self.fc_1(x)))        
        x = self.fc_2(x)
        
        return x

In [21]:
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()
        
        self.device = device        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)        
        self.dropout = nn.Dropout(dropout)        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        output = self.fc_out(trg)
            
        return output, attention
    
class DecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):

        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
        _trg = self.positionwise_feedforward(trg)
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        return trg, attention
    
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        return src_mask
    
    def make_trg_mask(self, trg):
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)        
        trg_len = trg.shape[1]        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        return trg_mask

    def forward(self, src, trg):
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)        
        enc_src = self.encoder(src, src_mask)                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        return output, attention

In [22]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 25,107,521 trainable parameters


In [23]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)
        
model.apply(initialize_weights);

In [24]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [25]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg        
        optimizer.zero_grad()        
        output, _ = model(src, trg[:,:-1])                            
        output_dim = output.shape[-1]            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)            
        loss = criterion(output, trg)        
        loss.backward()        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)        
        optimizer.step()        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    
    model.eval()    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg
            output, _ = model(src, trg[:,:-1])            
            output_dim = output.shape[-1]            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)            
            loss = criterion(output, trg)
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [26]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [27]:
N_EPOCHS = 5
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    test_loss = evaluate(model, test_iterator, criterion)
    torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Test. Loss: {test_loss:.3f} |  Test. PPL: {math.exp(test_loss):7.3f}')

Epoch: 01 | Time: 3m 42s
	Train Loss: 2.582 | Train PPL:  13.227
	 Test. Loss: 1.760 |  Test. PPL:   5.814
Epoch: 02 | Time: 3m 42s
	Train Loss: 1.604 | Train PPL:   4.970
	 Test. Loss: 1.453 |  Test. PPL:   4.278
Epoch: 03 | Time: 3m 43s
	Train Loss: 1.313 | Train PPL:   3.716
	 Test. Loss: 1.323 |  Test. PPL:   3.756
Epoch: 04 | Time: 3m 43s
	Train Loss: 1.146 | Train PPL:   3.146
	 Test. Loss: 1.254 |  Test. PPL:   3.506
Epoch: 05 | Time: 3m 42s
	Train Loss: 1.033 | Train PPL:   2.809
	 Test. Loss: 1.207 |  Test. PPL:   3.343


# Inference

In [28]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):
    
    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('de_core_news_sm')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    
    src_mask = model.make_src_mask(src_tensor)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attention

# Input data processing, inference

In [29]:
def basic_clean_ko(text):
    text = re.split('(\d+)', text) # 숫자와 글자 분리
    return ' '.join(text)

def sent_tok_korean(text):
   # text = re.sub(r'[^\w\s]', '', text) # 구두점 제거 
    text = text.replace('  ', ' ') # 다중 공백 제거
    text = re.split('([가-힣]+[^\w\s])', text) # 구두점과 글자 분리    
    return ''.join(text) # 단어 분리 

In [30]:
## 형태소 분석을 할 경우

sen_list_morph = [
'모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .',
'미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?',
'은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요',
'아무래도 분실 하 ㄴ 것 같 으니 분실 신고서 를 작성 하 아야 하 겠 습니다 . 사무실 로 같이 가 시 ㄹ 까요 ?',
'부산 에서 코로나 확진자 가 급증 하 아서 병상 이 부족하 아 지자  확진자 20명 을 대구 로 이송하 ㄴ다 .',
'변기 가 막히 었 습니다 .',
'그 바지 좀 보이 어 주 시 ㅂ시오 . 이거 얼마 에 사 ㄹ 수 있 는 것 이 ㅂ니까 ?',
'비 가 오 아서 백화점 으로 가지 말 고 두타 로 가 았 으면 좋 겠 습니다 .',
'속 이 안 좋 을 때 는 죽 이나 미음 으로 아침 을 대신 하 ㅂ니다',
'문 대통령 은 집단 이익 에서 벗어 나 아 라고 말 하 었 다 .',
'이것 좀 먹어 보 ㄹ 몇 일 간 의 시간 을 주 시 어요 .',
'이날 개미군단 은 외인 의 물량 을 모두 받 아 내 었 다 .',
'통합 우승 의 목표 를 달성하 ㄴ NC 다이노스 나성범 이 메이저리그 진출 이라는 또 다른 꿈 을 향하 어 나아가 ㄴ다 .',
'이번 구조 조정 이 제품 을 효과 적 으로 개발 하 고 판매 하 기 위하 ㄴ 회사 의 능력 강화 조처 이 ㅁ 을 이해 하 아 주 시 리라 생각 하 ㅂ니다 .',
'요즘 이 프로그램 녹화 하 며 많은 걸 느끼 ㄴ다 '
'배낭 여행 은 우리 들 이 어리었을 때 허용 되 지 않 으면 우울 하 아 하 ㄴ다'
'그 소녀 는 단지 늑대 처럼 울부짖 을 수 있 을 뿐 이 었 다']

# 데이터프레임으로 만들기
sent_morph = pd.DataFrame(sen_list_morph, columns = ["sentences"])
sent_morph["clean_Korean"] = sent_morph["sentences"].apply(basic_clean_ko)
sent_morph["clean_Korean"] = sent_morph["clean_Korean"].apply(sent_tok_korean)
sent_morph = sent_morph["clean_Korean"]
sent_morph.to_csv("sent_morph")

In [31]:
reading_sent = pd.read_csv("./sent_morph")
reading_sent["trg"] = "" # 빈 컬럼 만들어주기

In [32]:
Raw_datafields = [("Unnamed:0", None), ("src", SRC), ("trg", TRG)]

sent_data = data.TabularDataset(
        path='./sent_morph',
        format='csv',
        skip_header=True, 
        fields=Raw_datafields) 

SRC.build_vocab(train_data, min_freq = 1)
TRG.build_vocab(train_data, min_freq = 1)

In [33]:
pred_sent = []

for i in range(len(sent_data)):
    src = vars(sent_data[i])['src']
    translation, attention = translate_sentence(src, SRC, TRG, model, device)
    translation = ' '.join(translation)
    pred_sent.append(translation)
    
pred = pd.DataFrame(pred_sent, columns = ["pred_sentences"])

reading_sent["trg"] = pred["pred_sentences"]
reading_sent

,Unnamed: 0,clean_Korean,trg
0,0,"모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .","we have to bring all liquids , quart-size , quart-size , quart-size , quart-size , single plastic bag , quart-size , zip-top , zip-top , zip-top , zip-top , zip-top , zip-top , zip-top , zip-top , plastic , plastic , plastic , plastic , zip-top ,..."
1,1,"미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?","sorry , but the noise from being cold . could you change my ticket to gwanghwamun ? i am trying to move to move to move to another breeze passing the passenger to crazy ? <eos>"
2,2,은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요,the bank is too far to steal . i need a cash refund . <eos>
3,3,아무래도 분실 하 ㄴ 것 같 으니 분실 신고서 를 작성 하 아야 하 겠 습니다 . 사무실 로 같이 가 시 ㄹ 까요 ?,i think i have lost my report . would you mind checking out ? i am afraid i have to the office . <eos>
4,4,부산 에서 코로나 확진자 가 급증 하 아서 병상 이 부족하 아 지자 확진자 20 명 을 대구 로 이송하 ㄴ다 .,busan is a great skier with the pusan and fierce city hall and fierce will be bimonthly . <eos>
5,5,변기 가 막히 었 습니다 .,the toilet is clogged up . <eos>
6,6,그 바지 좀 보이 어 주 시 ㅂ시오 . 이거 얼마 에 사 ㄹ 수 있 는 것 이 ㅂ니까 ?,i would like to see them out . how much will it be ? <eos>
7,7,비 가 오 아서 백화점 으로 가지 말 고 두타 로 가 았 으면 좋 겠 습니다 .,i think we should go to the mall and go to the mall immediately . <eos>
8,8,속 이 안 좋 을 때 는 죽 이나 미음 으로 아침 을 대신 하 ㅂ니다,i feel frustrated because it tastes strange in sick with soup . <eos>
9,9,문 대통령 은 집단 이익 에서 벗어 나 아 라고 말 하 었 다 .,the door was made at the president . <eos>


In [34]:
## 어절단위로 결합하여 할 경우

sen_list_eojeol = [
'모든 액체, 젤, 에어로졸 등은 1커트 짜리 여닫이 투명 봉지 하나에 넣어야 합니다.',
'미안하지만 , 뒷쪽 아이들의 떠드는 소리가 커서 , 광화문으로 가고 싶은데 표를 바꾸어 주시겠어요?',
'은행이 너무 멀어서 안되겠네요. 현찰이 필요하면 돈을 훔치세요',
'아무래도 분실한 것 같으니 분실신고서를 작성해야 하겠습니다. 사무실로 같이 가실까요?',
'부산에서 코로나 확진자가 급증해서 병상이 부족해지자  확진자 20명을 대구로 이송한다.',
'변기가 막히었습니다 .',
'그 바지 좀 보여주십시오. 이거 얼마에 살 수 있는 것입니까?',
'비가 와서 백화점으로 가지 말고 두타로 갔으면 좋겠습니다 .',
'속이 안좋을 때는 죽이나 미음으로 아침을 대신 합니다',
'문 대통령은 집단이익에서 벗어나라고 말하였다.',
'이것 좀 먹어 볼 몇 일 간의 시간을 주세요.',
'이날 개미군단은 외인의 물량을 모두 받아내었다.',
'통합 우승의 목표를 달성한 NC 다이노스 나성범이 메이저리그 진출이라는 또다른 꿈을 향해 나아간다.',
'이번 구조 조정이 제품을 효과적으로 개발하고 판매하기 위한 회사의 능력 강화 조처임을 이해해 주시리라 생각합니다.',
'요즘 이 프로그램 녹화하며 많은 걸 느낀다'
'배낭 여행은 우리들이 어렸을 때 허용디지 않으면 우울 해 한다'
'그 소녀는 단지 늑대처럼 울부짖을 수 있을 뿐이었다']
sent_eojeol = pd.DataFrame(sen_list_eojeol, columns = ["sentences"])
sent_eojeol["clean_Korean"] = sent_eojeol["sentences"].apply(basic_clean_ko)
sent_eojeol["clean_Korean"] = sent_eojeol["clean_Korean"].apply(sent_tok_korean)
sent_eojeol = sent_eojeol["clean_Korean"]
sent_eojeol.to_csv("sent_eojeol")

reading_sent2 = pd.read_csv("./sent_eojeol")
reading_sent2["trg"] = ""

In [35]:
Raw_datafields = [("Unnamed:0", None), ("src", SRC), ("trg", TRG)]

sent_data2 = data.TabularDataset(
        path='./sent_eojeol',
        format='csv',
        skip_header=True, 
        fields=Raw_datafields) 

SRC.build_vocab(train_data, min_freq = 1)
TRG.build_vocab(train_data, min_freq = 1)

In [36]:
pred_sent = []

for i in range(len(sent_data2)):
    src = vars(sent_data2[i])['src']
    translation, attention = translate_sentence(src, SRC, TRG, model, device)
    translation = ' '.join(translation)
    pred_sent.append(translation)
    
pred = pd.DataFrame(pred_sent, columns = ["pred_sentences"])

reading_sent2["trg"] = pred["pred_sentences"]
reading_sent2

,Unnamed: 0,clean_Korean,trg
0,0,"모든 액체, 젤, 에어로졸 등은 1 커트 짜리 여닫이 투명 봉지 하나에 넣어야 합니다.",all the passengers are allowed to carry into the main distribution rights without a single room . <eos>
1,1,"미안하지만 , 뒷쪽 아이들의 떠드는 소리가 커서 , 광화문으로 가고 싶은데 표를 바꾸어 주시겠어요?","minsok-chon is a king and fierce swim in the outdoor life and the dark , is the mice . <eos>"
2,2,은행이 너무 멀어서 안되겠네요. 현찰이 필요하면 돈을 훔치세요,the dealer in the curve like a curve . <eos>
3,3,아무래도 분실한 것 같으니 분실신고서를 작성해야 하겠습니다. 사무실로 같이 가실까요?,i think the dealer will be improvised . <eos>
4,4,부산에서 코로나 확진자가 급증해서 병상이 부족해지자 확진자 20 명을 대구로 이송한다.,the border between dongho great resorts for the elderly . <eos>
5,5,변기가 막히었습니다 .,<eos>
6,6,그 바지 좀 보여주십시오. 이거 얼마에 살 수 있는 것입니까?,the trousers can get a prize . <eos>
7,7,비가 와서 백화점으로 가지 말고 두타로 갔으면 좋겠습니다 .,the eagles on the plane is falling apart . <eos>
8,8,속이 안좋을 때는 죽이나 미음으로 아침을 대신 합니다,the dealer in second-hand shed gill . <eos>
9,9,문 대통령은 집단이익에서 벗어나라고 말하였다.,the door is on the subway . <eos>


In [37]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []
    
    for datum in data:
        
        src = vars(datum)['src']
        trg = vars(datum)['trg']
        
        pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len)
        
        #cut off <eos> token
        pred_trg = pred_trg[:-1]
        
        pred_trgs.append(pred_trg)
        trgs.append([trg])
        
    return bleu_score(pred_trgs, trgs)

In [38]:
bleu_score = calculate_bleu(test_data, SRC, TRG, model, device)

print(f'BLEU score = {bleu_score*100:.2f}')

BLEU score = 43.39


# 정리

BATCH_SIZE = 64

N-epoch : 5

Train_PPL : 2.809

Test_PPL : 3.343

BLEU score : 43.39